In [ ]:
import pandas as pd
import numpy as np
import time 
import datetime
import random

In [ ]:
# need to initialize KMODE

from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

!pip install --upgrade kmodes

from kmodes.kmodes import KModes

In [ ]:
class AnonymizeError(Exception):
    def __init__(self, message="Invalid Input"):
        self.message = message
        super().__init__(self.message)


In [ ]:
def Dataset_Validator(dataframe,nan_replace_int = 0,nan_replace_str = '',comma_replacer = '/'):
  global _df
  dataframe = dataframe.dropna(how='all')
  dataframe[CAT_COL] = dataframe[CAT_COL].applymap(str).fillna(nan_replace_str)
  # dataframe[CAT_COL] = dataframe[CAT_COL].applymap(str).fillna(nan_replace_str)
  dataframe[CAT_COL] = dataframe[CAT_COL].applymap(lambda x: x.replace(",",comma_replacer))
  dataframe[NUM_COL] = dataframe[NUM_COL].fillna(nan_replace_int).applymap(lambda x: numerical_validator(x,nan_replace_int))
  _df =  dataframe  

def numerical_validator(value,nan_replace_int):
  try:
    if(type(value) == int or type(value) == float):
      return value
    elif('-' in str(value) or '- ' in str(value) or ' -' in str(value)):
      elements = value.split('-')
      return (int(elements[0].strip()) + int(elements[1].strip()))//2
    else:
      return int(value)
  except ValueError:
    return nan_replace_int
  except Exception as e:
    if(_DEBUG):
      print(e)
    return nan_replace_int


In [ ]:
def validator(dataframe,QI_,SA_,CAT_INDEXES_):
  global IS_CAT,QI,SA
  err_msg_1 = "quasi identifiers are not in columns of the dataframe"
  err_msg_2 = "Sensitive Attributes are not in columns of the dataframe"
  err_msg_3 = "Sensitive Attributes cannot be a subset of quasi identifiers"
  err_msg_7 = "Categorical index list cannot be more than quasi identifiers"
  err_msg_4 = "Invalid Catergorical index"
  err_msg_5 = "Duplicate Value"
  err_msg_6 = "Expect argument as a list"
  columns = dataframe.columns
  if(type(QI_) != list):
    raise AnonymizeError(message = err_msg_6+" in "+ 'quasi identifiers')
  elif(type(SA_) != list):
    raise AnonymizeError(message = err_msg_6+" in "+ 'sensitive attributes')
  elif(type(CAT_INDEXES_) != list):
    raise AnonymizeError(message = err_msg_6+" in "+ 'catergorical index')
  elif(len(set(QI_)) != len(QI_)):
    raise AnonymizeError(message = err_msg_5+" in "+'quasi identifiers')
  elif(len(set(SA_)) != len(SA_)):
    raise AnonymizeError(message = err_msg_5+" in "+ 'sensitive attributes')
    
  elif not(set(QI_).issubset(set(columns))):
    raise AnonymizeError(message = err_msg_1)
  elif not(set(SA_).issubset(set(columns))):
    raise AnonymizeError(message = err_msg_2)
  elif(len(set(SA_).intersection(set(QI_))) > 0):
    raise AnonymizeError(message = err_msg_3)
  else:
    try:
      CAT_INDEXES_ = list(map(int,CAT_INDEXES_))
    except:
      raise AnonymizeError(message = err_msg_4+"\n Index should be a integer")
    CAT_INDEXES_sorted = sorted(CAT_INDEXES_)
    if(len(CAT_INDEXES_) != 0):
      if not(0 <= CAT_INDEXES_sorted[-1] < len(QI_)):
        raise AnonymizeError(message = err_msg_4+"\ncatergorical index should start with zero and in between 0 and number of quasi identifiers")
      if not(0 <= CAT_INDEXES_sorted[0] < len(QI_)):
        raise AnonymizeError(message = err_msg_4+"\ncatergorical index should start with zero and in between 0 and number of quasi identifiers")
    if(len(CAT_INDEXES_) > len(QI_)):
      raise AnonymizeError(message = err_msg_7)
    elif(len(set(CAT_INDEXES_)) != len(CAT_INDEXES_)):
      raise AnonymizeError(message = err_msg_5 +" in "+'Categorical Index')
    IS_CAT = [False]*len(QI_)
    try:
      if(len(CAT_INDEXES_) != 0):
        for index in CAT_INDEXES_:
          IS_CAT[index] = True
        print("\n")
        print(CAT_INDEXES_,IS_CAT)
        print("\n")
    except:
      raise AnonymizeError(message = "Invalid index for categorical indexes")
    QI = QI_
    SA = SA_
    marking_globals(dataframe)
    Dataset_Validator(_df)
    return True



In [ ]:
def marking_globals(df):
    global QI_LEN,QI,_df,SA,IS_CAT,QI_RANGE_VAL,QI_RANGE_VAL,CAT_UNIQUE,NUM_COL,CAT_COL,_DEBUG,RANGE_FIX,CAT_INDEXES
    QI_LEN = len(QI)
    CAT_INDEXES,CAT_COL,NUM_COL = [],[],[]
    drop_col = []
    CAT_UNIQUE = []
    _DEBUG = True
    QI_RANGE_VAL = []
    RANGE_FIX = 1
    if(_DEBUG):
      print("Starting initializing globals")
    for column in df.columns:
        if not((column in QI) | (column in SA)):
            drop_col.append(column)
    if(_DEBUG):
      print("After initializing drop column")
    df = df.drop(drop_col, axis=1)
    _df = df.loc[:,QI+SA]
    sensitive_input = df.loc[:,SA]
    if(_DEBUG):
      print("Before marking QI_RANGE_VAL")
    try:
      for i in range(QI_LEN):
          if(IS_CAT[i] is False):
              diff = df[QI[i]].max() - df[QI[i]].min()
              QI_RANGE_VAL.append(diff)
          else:
            unique_count = len(df[QI[i]].unique())
            CAT_UNIQUE.append(unique_count)
            QI_RANGE_VAL.append(unique_count)
    except:
      raise(AnonymizeError(message = "Invalid categorical index. Check whether catergorical indexes are correct"))
    if(_DEBUG):
      print("Before marking NUM_COL & CAT_COL & CAT_INDEXES")
    for i,ele in enumerate(IS_CAT):
        if(ele):
          CAT_COL.append(QI[i])
          CAT_INDEXES.append(i)
        else:
          NUM_COL.append(QI[i])
    if(_DEBUG):
        print("Finished initializing Globals")


In [ ]:
#### reducedata
'reduced_adult.csv'
'''
age,workcalss,education_num,matrital_status,occupation,race,sex,native_country
class
False,True,False,True,True,True,True,True
'''

#### adult.data
'adult.data'
'''
age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
age,workclass,education-num,marital-status,occupation,race,sex,native-country
income
False,True,False,True,True,True,True,True
'''

'\nage,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income\nage,workclass,education-num,marital-status,occupation,race,sex,native-country\nincome\nFalse,True,False,True,True,True,True,True\n'

In [ ]:
######### Define variables to make easy for debugging ###################
global row_number
row_number = 0


In [ ]:
##########   Cluster related methods #################


def edit_cluster(group,distances):
    '''
    This method is used to edit the cluster number of a record.
    group :- Cluster group, type = Pandas DataFrameGroupBy
    distances :- Dataframe that include cluster mapping details
    '''
    c_no = group['cluster_number'].unique()[0]
    group['cluster_number'] = distances[c_no]
    return group

def numeric_range(numeric_values):
  '''
  This method is used to generalize the data in a cluster.
  The given values in a cluster turn to the min-max range if they are not same.
  numeric_values :- Numeric values of the cluster. type = Pandas DataFrame
  '''
  numerical_min = numeric_values.min()
  numerical_max = numeric_values.max()
  return numerical_min.where(numerical_min.eq(numerical_max),numerical_min.apply(str) +"-"+numerical_max.apply(str))

def catergorical_range(catergorical):
  '''
  This method is used to anonymize catergorical values in a cluster.
  Given categorical values turn to comma seperated generalization if they are not same.
  catergorical :- Catergorical values in a cluster. type = Pandas DataFrame
  '''
  catergorical = catergorical.apply(str)
  return ','.join(catergorical.unique())

def anonimize_catergorical_value_in_cluster(frame):
  '''
  This method is used to anonymize catergorical values in a cluster
  Given categorical values turn to comma seperated generalization if they are not same.
  frame :- Catergorical values in a cluster. type = Pandas DataFrame
  '''
  return str(','.join(frame.unique()))



In [ ]:
###################################  Distance Finding ###########################

def cal_num_col_dist(record,cluster_record,range_value,factor):
    '''
    This method is used to find the numerical distance between records and cluster genaralization values
    record :-  The record need to find the distance with cluster  type :- Pandas Series
    cluster_record :- Numerical generalization values of the clusters   type :- Pandas DataFrame
    range_value :- Clusters range values   type :- Pandas DataFrame
    factor :- factor need to multiply. Recommended 20. type:- Integer
    '''
    return abs((cluster_record - record)/range_value*factor)

def cal_cat_col_dist1(row,categorical_col):
    '''
    This method is used to find the categorical distance between records and cluster genaralization values
    row :-  The record need to find the distance with cluster  type :- Pandas Series
    categorical_col :- Catergorical generalization values of the clusters   type :- Pandas DataFrame
    '''
    return (len(CAT_COL)-np.where(
        row[CAT_COL]==categorical_col,True,np.where(
            categorical_col == "*****",True,False)).sum(axis=1))*10

def cal_cat_col_dist2(row,categorical_col):
    '''
    This method is used to find the categorical distance between records and cluster genaralization values
    row :-  The record need to find the distance with cluster  type :- Pandas Series
    categorical_col :- Catergorical generalization values of the clusters   type :- Pandas DataFrame
    '''
    return (len(CAT_COL)-np.where(
        row[CAT_COL]==categorical_col,1,np.where(
            categorical_col == "*****",0.5,0)).sum(axis=1))*10


def cal_cat_col_dist3(row,categorical_col):
    '''
    This method is used to find the categorical distance between records and cluster genaralization values
    row :-  The record need to find the distance with cluster  type :- Pandas Series
    categorical_col :- Catergorical generalization values of the clusters   type :- Pandas DataFrame
    '''
    return (len(CAT_COL)-np.where(
        row[CAT_COL]==categorical_col,1,0).sum(axis=1))*10



In [ ]:
###########Kmode methods

def find_distance_kmode_clusters(row,centroid,range_value):
  '''
  This method is used to find the ditance from records to cluster genaralization record in KMODE clustering
  row :-  The record need to find the distance with cluster  type :- Pandas Series
  centroid :- Generalization values of the clusters   type :- Pandas DataFrame
  range_value :- Ranges for numerical values type :- Pandas DataFrame
  '''
  return np.sum((row[CAT_COL]!=centroid[CAT_COL]).tolist())*10 + np.sum(abs((centroid[NUM_COL] - row[NUM_COL])/range_value*10))

In [ ]:
########### cluster dataloss related methods ####################

def numerical_dataloss(cluster,cluster_list,range_value,factor=20):
    '''
    This function used to find the numerical dataloss in a situation of two clusters join
    cluster :- Numerical values in the cluster type:-Pandas DataFrameGroupBy
    cluster_list :- Numerical values in the groups of clusters  type:-Pandas DataFrameGroupBy
    range_value :- Ranges for numerical values type :- Pandas DataFrame
    factor :- factor need to multiply. Recommended 20. type:- Integer
    '''
    return np.sum(((abs(cluster_list.min() - cluster.min()))+abs(cluster_list.max() - cluster.max()))/range_value*factor,axis=1)


def get_uniques(rows,cluster):
    '''
    This method is used to compare unique catergorical values in two clusters.
    rows :- Categorical values in the first cluster. type :- Pandas DataFrame
    cluster :- Catergorical values in second cluster. type :- Pandas DataFrame
    '''
    row_unique =[]
    cluster_unique = []
    conc_arr = []
    val = 0
    for i,col in enumerate(CAT_COL):
        row_num = np.array(rows[col].unique())
        cluster_num = np.array(cluster[col].unique())
        conc_arr = np.intersect1d(row_num,cluster_num)
        val+= (row_num.size + cluster_num.size - 2*(conc_arr.size))
    return val

def get_uniques1(row,cluster):
    '''
    This method is used to compare unique catergorical values in two clusters.
    rows :- Categorical values in the first cluster. type :- Pandas Dataframe
    cluster :- Catergorical values in second cluster. type :- Pandas Dataframe
    '''
    frame = pd.concat([row,cluster])
    if(row.equals(cluster)):
        return 0
    return np.sum([(len(frame[col].unique()) -1)/CAT_UNIQUE[i] for i,col in enumerate(CAT_COL)])

def categorical_dataloss1(cluster,cluster_list,factor=20):
    '''
    This method is used to compare unique catergorical values in two clusters.
    cluster :- Categorical values in the cluster. type :- Pandas DataFrame
    cluster_list :- Catergorical values in cluster values. type :- Pandas DataFrameGroupBy
    factor :- Factor to multiply the dataloss. Recomonded 20. Type Integer
    '''
    return cluster_list.apply(lambda row: get_uniques(row,cluster))

def categorical_dataloss(cluster,cluster_list,factor=20):
    '''
    This method is used to compare unique catergorical values in two clusters.
    cluster :- Categorical values in the cluster. type :- numpy array
    cluster_list :- Catergorical values in cluster values. type :- Pandas numpy array
    factor :- Factor to multiply the dataloss. Recomonded 20. Type Integer
    '''
    cluster = cluster.reshape((1,cluster.shape[0]))
    cluster_list = cluster_list.reshape((1,cluster_list.shape[0]))
    intersection_list = np.intersect1d(cluster[0],cluster_list[0])
    return abs(cluster[0].size+cluster_list[0].size-2*intersection_list.size)
categorical_dataloss = np.vectorize(categorical_dataloss,excluded="cluster_list")


In [ ]:
class Kanonymizer(object):
    def __init__(self,df,QI_attr,Sensitive_attr,Cat_index,verbose = 1,max_iter = 10,max_cluster_distance = 20):
        '''
        
        Attributes:-
          df :- Pandas Dataframe that will used to anonymize. Type :- Pandas DataFrame
          QI_attr :- Column names of Quasi Identifiers. Type :- list
          Sensitive_attr :- Column names of Sensitive Columns
          verbose : Log details(1) or not(0).Default value is 1. Type : Boolean
          max_iter :- The maximum iteration number of joining clusters. Default value is 5 .Type :- Integer
          max_cluster_distance :- The maximum value in cluster distance. Default value is 20. Type :- Interger
        '''
        validation = validator(df,QI_attr,Sensitive_attr,Cat_index)
        self.df = df.copy()
        self.df_copy = self.df.copy()
        self.df_second_copy = self.df.copy()
        self.QI_attr = QI_attr
        self.Sensitive_attr = Sensitive_attr
        self.n_clusters = 0
        self.verbose = verbose
        self.centroids = None
        self.less_centroids = None
        self.k_centroids = None
        self.k = 0
        self.max_iter = max_iter
        self.anonimize_ratio = 1
        self.max_cluster_distance = max_cluster_distance
        self.cluster_distances = None

########### Methods to write pandas dataframe to a file  #########################
 
    def file_write(self,file_name = 'output.csv',sep_ = ',',encoding_ = 'utf-8'):
      self.df[QI+SA].to_csv(file_name, sep=sep_, encoding=encoding_)

##################################################################################

###### Related methods for equal clustering option

    def _level_cluster(self,cluster_num):
        if(_DEBUG):
            print("In _level_cluster")
        cluster_len = self.df.loc[self.df['cluster_number']==cluster_num].shape[0]
        additional_entries = self.df.loc[self.df['cluster_number']==cluster_num].nlargest(cluster_len-self.k,'cluster_distance')
        self.df.at[additional_entries.index,'cluster_number'] = -1
        self.df.at[additional_entries.index,'cluster_distance'] = -1

    def _adjust_big_clusters1(self):
        if(_DEBUG):
            print("In _adjust_big_clusters1")
        temp = self.df['cluster_number'].value_counts()
        big_clusters = temp.loc[temp > self.k].index
        for index in big_clusters:
            self._level_cluster(index)

    def _adjust_small_clusters1(self):
        if(_DEBUG):
            print("In _adjust_small_clusters1")
        num_of_iter = 0
        while(not(self.df['cluster_number'] != -1).all()):
            num_of_iter += 1
            # print(self.df.loc[self.df['cluster_number']==-1])
            best_clusters = (self.df.loc[self.df['cluster_number']==-1]).apply(lambda row: self._clustering2(row),axis=1)
            self.df.at[best_clusters.index,'cluster_number'] = best_clusters.values
            if(_DEBUG):
              print("Going to adjust big clusters after small adjust")
            self._adjust_big_clusters1() 
            if(num_of_iter >= self.max_iter ):
                self.df = self.df.loc[self.df['cluster_number'] != -1]
                break


    def _find_best_cluster(self,row,clusters,factor= 20):
        """
        calcuate the distance from a data point to all the cluster centroids
        and find the best cluster
        
        row - datapoint as pandas series from the dataframe, 
        clusters - cluster dataframe with all the cluster centroids
        factor - is to change priority between numerical data points and categorical data points
        when you give a higher value for the factor numerical data points get higher priority 
        resulting lower ranges for a cluster. but this cause higher data loss for categorical data

        returns the index/cluster_number that best fit to the given row.
        """
        if(_DEBUG):
            print("In _find_best_cluster")
        numerical_col = clusters[NUM_COL]
        categorical_col = clusters[CAT_COL]
        ranges = numerical_col.max() - numerical_col.min()
        # print(numerical_col)
        # print(ranges)
        # print(self.df['native_country'])
        numerical_distance = cal_num_col_dist(row[NUM_COL],numerical_col,ranges,factor)
        if(_DEBUG):
            print("In _find_best_cluster After numerical distancing")
        categorical_distance = cal_cat_col_dist3(row,categorical_col)
        
        if(_DEBUG):
            print("In _find_best_cluster After distancing")
        # if categorical_col.empty and _DEBUG:
        #   print(clusters)
        #   print("eow",row)
          # print('categorical_distance',categorical_distance,numerical_distance.sum(axis=1))
          # print('DataFrame is empty!'*100)
        self.df.at[row.name,'cluster_distance'] = (numerical_distance.sum(axis=1)+categorical_distance).min()
        return (numerical_distance.sum(axis=1)+categorical_distance).idxmin()
    
    def _clustering1(self, row):
        global row_number
        row_number+=1
        if(_DEBUG):
            print(row_number)
            print("In _clustering1")
        best_cluster = self._find_best_cluster(row,self.centroids)
        return best_cluster

    def _clustering2(self,row):
        temp = self.df['cluster_number'].value_counts()
        # print(temp)
        small_clusters = temp.loc[temp < self.k].index
        if (-1 in small_clusters):
            small_clusters.drop(-1)
        best_cluster = self._find_best_cluster(row,self.centroids.iloc[small_clusters])
        return best_cluster

############## Methods for KMODE clustering

    def komode_clustering(self,catergorical_indexes = [],type_ = 'random',n_init_=3,verbose_=0):

        '''
        This method is used to clustering based on KMODE clustering.

        dataset : The pandas dataframe want to cluster. Type :- Pandas DataFrame
        catergorical_indexes = index of columns with catergorical type data. Type List 
        type_ = Method to clustering - Hung or Cao or random. Type String
        n_clusters_ = number of clusters. Type :- Integer 
        n_init_ = number of iterations to compare. Type :- Integer 
        verbose : Log details(1) or not(0). Type : Boolean
        '''
        km = KModes(self.n_clusters, init=type_, n_init=n_init_, verbose=verbose_)
        y=km.fit_predict(self.df[QI],categorical=catergorical_indexes)
        # print(km.cost_)
        # print(km.epoch_costs_ )
        # print(km.cluster_centroids_)
        # print(km.labels_)
        columns = pd.DataFrame(km.cluster_centroids_,columns = QI)
        
        columns[NUM_COL] = columns[NUM_COL].applymap(lambda x: np.float(x))
        self.df[NUM_COL] = self.df[NUM_COL].applymap(lambda x: np.float(x))
        columns[CAT_COL] = columns[CAT_COL].applymap(lambda x: np.str(x))
        self.df[CAT_COL] = self.df[CAT_COL].applymap(lambda x: np.str(x))
        self.df['cluster_number'] = list(km.labels_)
        non_zero_member_cluster_indices = self.df.groupby('cluster_number').filter(lambda grp : len(grp) != 0)['cluster_number'].unique()
        columns = columns.loc[non_zero_member_cluster_indices]
        columns = columns.reset_index()
        index_series = pd.Series(columns.index,index = non_zero_member_cluster_indices)
        self.c_centroids = columns
        self.df['cluster_number'] = self.df.apply(lambda row:index_series.loc[row['cluster_number']],axis = 1)
        if(_DEBUG):
            print("COLUMNS"*100)
            print(columns)
            print("list(km.labels_)"*100)
            print(list(km.labels_))
        return self.df


############ Methods for initializing centroids ####################

    def find_best_cluster_gens(self,dataframe):
        '''
        This method return cluster centroids weight on the probability of row's column values appear in dataframe.
        dataframe :- Unique rows in the dataframe. Type :- Pandas Dataframe
        '''
        length = len(dataframe)
        if(_DEBUG):
          print("Before counting prob")
        probs = [dataframe[column].value_counts()/length  for column in QI]
        if(_DEBUG):
          print("After counting prob")
        dataframe['probability_sum'] = dataframe[QI].apply(lambda row: np.sum([probs[i][row[column]] for i,column in enumerate(QI)]),axis = 1)
        points = dataframe.sample(n = self.n_clusters , weights = 'probability_sum')
        dataframe = dataframe[QI]
        return points[QI]

    def select_centroids_using_weighted_column(self,dataframe):

        '''
        This method will choose centroids weight according to the column that has most number of unique values.
        dataframe :- Unique rows in the dataframe. Type :- Pandas Dataframe
        '''
        counts = [len(dataframe[column].unique())  for column in QI]
        column_counts = sorted(counts)
        column = counts.index(column_counts[-1])
        try:
          return dataframe.sample(n = self.n_clusters , weights = QI[column])
        except Exception:
          return self.find_best_cluster_gens(dataframe)

    def random_sample_centroids(self,unique_rows):
        '''
        Return cluster centroids in randomly.
        dataframe :- Unique rows in the dataframe. Type :- Pandas Dataframe
        '''
        return unique_rows.sample(n=self.n_clusters)


########### Methods for removing and marking less clusters #############################

    def remove_far_clusters(self):
        '''
        This method is used to remove the far away clusters according to users preference
        Class attribute anonimize_ratio used to measure the distance
        '''
        temp = self.df['cluster_number'].value_counts()
        very_small_clusters = temp.loc[temp <= self.k*self.anonimize_ratio].index
        not_small_cluster = self.df.apply(lambda row: row['cluster_number'] not in very_small_clusters,axis=1)
        self.centroids = self.centroids.iloc[temp.loc[temp > self.k*self.anonimize_ratio].index]
        self.df = self.df.loc[not_small_cluster]
        if(_DEBUG):
            print(self.df)
            print(self.centroids)
        return self.df.sort_values(by=['cluster_number']),self.centroids

    def mark_less_n_kcentroids(self,dataframe = 'second'):
        '''
        This method is used to mark cluster centroids which has less than k number of members.
        dataframe :- The dataframe is used to couunt the cluster members. Type Pandas Dataframe
        '''
        if(dataframe == 'second'):
          dataframe = self.df_second_copy
        else:
          dataframe = self.df
        temp = dataframe['cluster_number'].value_counts()
        self.less_centroids = self.centroids.loc[temp.loc[temp < self.k*self.anonimize_ratio].index]
        self.k_centroids = self.centroids.loc[temp.loc[temp >= self.k*self.anonimize_ratio].index]
        if(_DEBUG):
          print("\n")
          print(self.less_centroids)
          print(self.k_centroids)
          print("\n")

########### measure centroid distance between centroids  ###################

    def get_distance_centers(self,cluster_):
        '''
        This method is used to find the distances between cluster generalization values.
        cluster_ :- The cluster generalization value that need to find the distance. Type :- Pandas Series
        '''
        categorical_col = self.centroids[CAT_COL]
        numerical_col = self.centroids[NUM_COL]
        ranges = numerical_col.max() - numerical_col.min()
        return np.sum(cal_num_col_dist(cluster_[NUM_COL],numerical_col,ranges,20),axis=1) + cal_cat_col_dist3(cluster_,categorical_col)

######### Finding the data loss of joining two clusters ###################

    def cluster_data_loss(self,apply_for = 'less_clusters',initialize=True):
        '''
        This method is used to find the dataloss of joining two clusters.
        This function return the dataloss among each and every clusters
        apply_for :- If this is 'less_clusters', Dataloss will find only for clusters that have less members than k. Type :- String
        initialize :- This parameter define Is it essential to initialize k less clusters or not. Type :- Boolean
        '''
        if(_DEBUG):
          print("cluster_data_loss")
        if(initialize):
            self.mark_less_n_kcentroids()
        self.less_centroids.sort_index(inplace=True)
        center_groups = self.df.groupby('cluster_number')
        if(_DEBUG):
          print("Above to go to cen_num")
        center_num = center_groups[NUM_COL]
        if(_DEBUG):
          print("Above to go to cen_cat")
        center_cat = center_groups[CAT_COL]
        if(_DEBUG):
          print("Above to go to groups")
        # groups = center_groups.apply(lambda x: np.unique(np.concatenate(x[CAT_COL].values)))
        groups = center_groups.apply(lambda x: np.unique(np.concatenate(x[CAT_COL].values).astype(str)))
        if(_DEBUG):
          print("Above to go to numpy array")
        groups = np.array(groups)
        groups = groups.reshape((groups.shape[0],1))
        if(_DEBUG):
          print("Above to go to ranges")
        ranges = center_num.max() - center_num.min() +RANGE_FIX
        if(_DEBUG):
          print("After center_groups")
        if(apply_for == 'less_clusters'):
            less_groups = self.df.groupby('cluster_number').filter(lambda x: len(x)<self.k).groupby('cluster_number')
            print(less_groups.ngroups)
            if(less_groups.ngroups == 0):
              return None
            less_lists = less_groups.apply(lambda x: np.unique(np.concatenate(x[CAT_COL].values).astype(str)))
            less_lists = np.array(less_lists)
            less_lists = less_lists.reshape((less_lists.shape[0],1))
            if(_DEBUG):
              print("less_group")
            cat_distances =  np.apply_along_axis(categorical_dataloss,1,less_lists,groups)
            num_distance =  less_groups.apply(lambda row:numerical_dataloss(row[NUM_COL],center_num,ranges))
            cat_frame_indices = self.less_centroids.index
        else:
            cat_distances =  np.apply_along_axis(categorical_dataloss,1,groups,groups)
            num_distance =  center_groups.apply(lambda row:numerical_dataloss(row[NUM_COL],center_num,ranges))
            cat_frame_indices = self.centroids.index
        shape = cat_distances.shape
        cat_distances = cat_distances.reshape(shape[0],shape[1]*shape[2])
        cat_frame = pd.DataFrame(cat_distances,index = cat_frame_indices)
        return cat_frame.add(num_distance, fill_value=QI_LEN*self.max_cluster_distance)

############# Kless clusters adjusting   ########################################################3333

    def adjust_kless_clusters(self,n = 2,method='dataloss'):
        '''
        This method is used to anonymize clusters that have less than k members.
        n :- Define the number of clusters that will join with a single less member cluster. Type :- Integer
        method :- If this is dataloss, Distance of two clusters is measured by the dataloss of joining, else Distance between cluster centroids. Type :- String
        '''
        self.mark_less_n_kcentroids()
        if(_DEBUG):
          print("marked_less clusters")
        self.df_copy = self.df.copy()
        self.df_copy['is_anonimized'] = 0
        if(method == 'dataloss'):
            centroid_distances = self.cluster_data_loss()
        else: 
            centroid_distances = self.less_centroids.apply(lambda row : self.get_distance_centers(row),axis=1)
        if(_DEBUG):
          print("Get cenr distance")
        n_close_centroids = np.argsort(centroid_distances,axis=1).iloc[:,:n]
        if(_DEBUG):
          print("After taking closing")
        results =  self.less_centroids.apply(lambda row: self.anonimize_k_less_clusters(row,n_close_centroids),axis=1)
        return results

    def anonimize_k_less_clusters(self,centroid,n_close_centroids):
        '''
        This method used to anonymize a single less member cluster
        centroid:- The centroid of the cluster. Type :- Pandas Series
        n_close_centroids :- This have the details about distances between cluster.(Distances measure by dataloss or centroid distances). Type :- Pandas DataFrame        
        '''
        if(_DEBUG):
          print("anonimize_k_less_clusters")


        close_cetroids = n_close_centroids.loc[centroid.name]
        if(_DEBUG):
          print("take_n_close")
        centroid_related_rows = self.df.loc[self.df.apply(lambda x:x['cluster_number'] in close_cetroids.values,axis=1)].sort_values(by=['cluster_number'])
        if(_DEBUG):
          print("centroid_related_rows")
        related_indexes = self.df_copy.apply(lambda x: x['cluster_number'] == centroid.name,axis=1)
        if(_DEBUG):
          print("related_indexes")
        numerical_min = centroid_related_rows[NUM_COL].min()
        numerical_max = centroid_related_rows[NUM_COL].max()
        centroid_related_rows[NUM_COL] = np.where(numerical_min == numerical_max,numerical_min,numerical_min.apply(str) +"-"+numerical_max.apply(str))
        centroid_related_rows[CAT_COL] = np.where(centroid_related_rows[CAT_COL].nunique() == 1,centroid_related_rows[CAT_COL],centroid_related_rows[CAT_COL].apply(anonimize_catergorical_value_in_cluster))
        centroid_related_rows = centroid_related_rows.loc[related_indexes]
        if(_DEBUG):
          print("centroid_related_rows")
        # self.df_copy.at[self.df_copy.loc[related_indexes].index,'is_anonimized'] = 1
        self.df_copy.at[related_indexes,'is_anonimized'] = 1
        self.df_copy.at[related_indexes,QI] = centroid_related_rows[QI]
        if(_DEBUG):
          print("above to finish k less")
        return centroid_related_rows


########## mark and join clusters ###################
    def make_anonymize(self,method = 'dataloss'):
        '''
        This method is used to generalize the dataframe after clustering
        method :- If this is dataloss, Distance of two clusters is measured by the dataloss of joining, else Distance between cluster centroids. Type :- String
        '''       
        result = self.mark_clusters(method = 'dataloss')
        if(_DEBUG):
          print("final result",result)
        if(result == 1):
          return 1
        else:
          self.mark_less_clusters_to_kclusters()
          
          

    def mark_clusters(self,method = 'dataloss'):
        '''
        This method is used join less member clusters to nearest cluster.
        method :- If this is dataloss, Distance of two clusters is measured by the dataloss of joining, else Distance between cluster centroids. Type :- String
        '''
        self.df_second_copy = self.df.copy()
        if(method == 'dataloss'):
            self.cluster_distances = self.cluster_data_loss()
        else:
            self.cluster_distances = self.less_centroids.apply(lambda row : self.get_distance_centers(row),axis=1)
        iteration_num = 0
        while(True):
          less_groups = self.df_second_copy.groupby('cluster_number').filter(lambda x: len(x)<self.k).groupby('cluster_number')
          if(less_groups.ngroups == 0):
            return 1
          elif(iteration_num>= self.max_iter):
            return 0
          else:
            if(_DEBUG):
              self.mark_less_n_kcentroids()                
            # self.mark_less_clusters_to_close_clusters(self)
            self.mark_less_clusters_to_close_clusters()
            iteration_num+=1

    def mark_less_clusters_to_close_clusters(self,method = 'dataloss'):
        '''
        This method is used join less member clusters to nearest cluster.
        method :- If this is dataloss, Distance of two clusters is measured by the dataloss of joining, else Distance between cluster centroids. Type :- String
        '''
        self.mark_less_n_kcentroids()        
        try:
            n_close_centroids = np.argsort(self.cluster_distances,axis=1).iloc[:,1]            
        except IndexError:
            return 1            
        less_groups = self.df.groupby('cluster_number').filter(lambda x: len(x)<self.k)
        groups = less_groups.groupby('cluster_number')
        less_cluster_indices = less_groups.index
        
        self.df_second_copy.at[less_cluster_indices,'cluster_number'] = groups.apply(lambda grp: edit_cluster(grp,n_close_centroids))


    def mark_less_clusters_to_kclusters(self,method='dataloss'):
        '''
        This method is used join less member clusters to nearest k or more member cluster.
        method :- If this is dataloss, Distance of two clusters is measured by the dataloss of joining, else Distance between cluster centroids. Type :- String
        '''
        if(_DEBUG):
          print("Ready to join k")
        self.mark_less_n_kcentroids()
        k_indices = self.df_second_copy.groupby('cluster_number').filter(lambda x: len(x)>=self.k)['cluster_number'].unique()
        # k_clusters = 
        cluster_distances = self.cluster_distances

        # cluster_distances[cluster_distances.columns != k_indices] = self.max_cluster_distance*10
        cluster_distances = cluster_distances[k_indices]


        try:
            if(k_indices.size == 1):
              n_close_centroids = np.argsort(cluster_distances,axis=1).iloc[:,0]
            else:
              n_close_centroids = np.argsort(cluster_distances,axis=1).iloc[:,1]
            # n_close_centroids = np.argsort(cluster_distances,axis=1).iloc[:,1]
            cols = cluster_distances.columns
            n_close_centroids = n_close_centroids.apply(lambda row: cols[row])
            
        except IndexError:
            return 1
        less_groups = self.df_second_copy.groupby('cluster_number').filter(lambda x: len(x)<self.k)
        groups = less_groups.groupby('cluster_number')
        less_cluster_indices = less_groups.index
        
        self.df_second_copy.at[less_cluster_indices,'cluster_number'] = groups.apply(lambda grp: edit_cluster(grp,n_close_centroids))
        
        
    def anon_k_clusters(self):
        '''
        This method is used to generalize clusters.
        '''
        if(_DEBUG):
          print("Ready to anonimize")
        groups = self.df_second_copy.groupby('cluster_number')
        if(_DEBUG):
          print("Before apply num range")
        num_vals = groups[NUM_COL].apply(numeric_range)
        if(_DEBUG):
          print("Before apply cat range")
        cat_vals = groups[CAT_COL].apply(lambda row: row.apply(catergorical_range))
        if(_DEBUG):
          print("Before apply anom range")
        if(NUM_COL != []):
          anom_vals = num_vals.join(cat_vals)[QI]
        else:
          anom_vals = cat_vals.join(num_vals)[QI]
        if(_DEBUG):
          print("After apply anom range")
        self.df_second_copy[QI] = self.df_second_copy.apply(lambda row: anom_vals.loc[row['cluster_number']],axis=1)
        if(_DEBUG):
          print("Finish anonym")



################# Anonymization function ################
    def anonymize(self,k,mode='',center_type = 'fbcg',return_mode = 'Not_equal',iter=1):
        '''
        This method is used to anonymize the Dataset
        Parameters :- 
            k = Number of rows that cannot be distinguished from each other
            mode = if this is 'kmode', clustering will happen using KMODE clustering. Else it will happen in using Pandas Dataframe functions.
            center_type = Defines the method to choose cluster centers.
                  If method is not equal to kmode. Three values are possible.
                      1. 'fcbg' = This method return cluster centroids weight on the probability of row's column values appear in dataframe. Default Value.
                      2. 'rsc'  = This method will choose centroids weight according to the column that has most number of unique values.
                      3. 'random = Return cluster centroids in randomly.
            return_mode = If this value equal to 'equal' ; K anonymization will done with equal member clusters. Default value is 'Not_Equal'
        
        return :- 
            Anonymize dataset. Type :- Pandas DataFrame
        
        '''
        
        if(_DEBUG):
          print("in anonymize")

        self.k = k
        self.n_clusters = len(self.df)//k
        self.c_centroids = np.zeros(len(self.df))
        self.df = _df.copy()
        data = self.df.copy()
        unique_rows = data[QI].drop_duplicates()
        mode = mode.lower()
        center_type = center_type.lower()
        return_mode = return_mode.lower()
        if(mode != 'komde'):
            if(_DEBUG):
              print("Initializing centroidds")
            if(center_type == 'rsc'):
                self.centroids = self.random_sample_centroids(unique_rows)
            elif(center_type == 'random'):
                self.centroids = self.select_centroids_using_weighted_column(unique_rows)
            else:
                self.centroids = self.find_best_cluster_gens(unique_rows)
            self.centroids.reset_index(drop=True, inplace=True)
        
            if(_DEBUG):
              print("After centroid initializing")

        while(iter!=0):
            global row_number
            row_number = 0
            iter = iter-1
            if(mode != 'kmode'):
                self.df['cluster_number'] = self.df.apply(lambda row: self._clustering1(row),axis=1)
                if(return_mode == "equal"):
                    self._adjust_big_clusters1()
                    self._adjust_small_clusters1()
            else:
                if(center_type not in ['hung,cao']):
                    center_type = 'random'
                self.df = self.komode_clustering(catergorical_indexes = CAT_INDEXES,type_ = center_type,n_init_=self.max_iter,verbose_=self.verbose)
            self.make_anonymize()
            self.anon_k_clusters()
            if(_DEBUG):
              print("Before edit df")
            self.df[QI] = self.df_second_copy[QI]
            if(_DEBUG):
              print("Above to return")
            self.file_write()
            return self.df[QI+SA]
    

In [ ]:
names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
df = pd.read_csv('adult.data', sep=", ", header=None, names=names, index_col=False, engine='python');
df = df[0:1000]
qi = ['age', 'workclass','education-num', 'marital-status', 'occupation', 'race', 'native-country']
sa = ['income','sex']
ci = [1,3,4,5,6]
s = 5

print(qi+sa)
anonymizer = Kanonymizer(df.copy(),qi,sa,ci)
start = datetime.datetime.now()
df_ = anonymizer.anonymize(k = s,center_type = 'Cao',mode='kmode_',return_mode = 'equal')
print(datetime.datetime.now() - start)
frame = anonymizer.df_second_copy.sort_values(by=['cluster_number'])

Streaming output truncated to the last 5000 lines.
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
423
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
424
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
425
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
426
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
427
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
428
In _clustering1
In _find_best_cluster
In _find_best_cluster After numerical distancing
In _find_best_cluster After distancing
429
In _clustering1
In _find_best_cluster
In _find_best_clu